## Implement Data Monitors (Real-Time)

In [11]:
# Imports
import boto3
import botocore
import json
import os
from sagemaker import get_execution_role, Session
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.model_monitor import DefaultModelMonitor, CronExpressionGenerator, DataCaptureConfig
from datetime import datetime

In [12]:
# Initialize Session
session = Session()
role = get_execution_role()
region = session.boto_region_name

# Your Bucket and Paths
bucket = 'sagemaker-us-east-1-531690656306'
prefix = 'cardio_data'
baseline_results_uri = f's3://{bucket}/{prefix}/baseline-results'
monitor_output_uri = f's3://{bucket}/{prefix}/monitoring/reports'

# Use your deployed endpoint
endpoint_name = 'cardio-logistic-monitor-endpoint'

In [13]:
# Verify that endpoint exist
sm_client = boto3.client('sagemaker', region_name='us-east-1')

response = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print(f"Endpoint '{endpoint_name}' status: {status}")

Endpoint 'cardio-logistic-monitor-endpoint' status: InService


### Create Data Monitor Schedule

In [14]:
# Create Monitor object
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1200,
    sagemaker_session=session
)

In [15]:
# Generate unique schedule name (safe for reuse)
current_time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
schedule_name = f"cardio-data-monitor-schedule-{current_time}"

monitor.create_monitoring_schedule(
    monitor_schedule_name=schedule_name,
    endpoint_input=endpoint_name,
    output_s3_uri=monitor_output_uri,
    statistics=f'{baseline_results_uri}/statistics.json',
    constraints=f'{baseline_results_uri}/constraints.json',
    schedule_cron_expression=CronExpressionGenerator.daily(),
    enable_cloudwatch_metrics=True
)

print(f"Real-time Data Quality Monitoring Schedule '{schedule_name}' successfully created!")

Real-time Data Quality Monitoring Schedule 'cardio-data-monitor-schedule-2025-06-11-04-39-12' successfully created!


In [16]:
# Verify schedule exists
sm_client = boto3.client('sagemaker', region_name=region)
response = sm_client.list_monitoring_schedules()

print("\nActive Monitoring Schedules:")
for schedule in response['MonitoringScheduleSummaries']:
    print(schedule['MonitoringScheduleName'], ":", schedule['MonitoringScheduleStatus'])


Active Monitoring Schedules:
cardio-data-monitor-schedule-2025-06-11-04-39-12 : Pending
cardio-data-monitor-schedule-2025-06-11-04-37-53 : Scheduled
cardio-data-monitor-schedule-2025-06-11-03-31-48 : Scheduled


In [17]:
# Delete other data monitor schedules
sm_client = boto3.client("sagemaker", region_name="us-east-1")

# List of older schedules to delete
schedules_to_delete = [
    "cardio-data-monitor-schedule-2025-06-11-04-37-53",
    "cardio-data-monitor-schedule-2025-06-11-03-31-48"
]

for schedule_name in schedules_to_delete:
    try:
        sm_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)
        print(f"Deleted monitoring schedule: {schedule_name}")
    except Exception as e:
        print(f"Failed to delete {schedule_name}: {e}")

Deleted monitoring schedule: cardio-data-monitor-schedule-2025-06-11-04-37-53
Deleted monitoring schedule: cardio-data-monitor-schedule-2025-06-11-03-31-48


-----

## Infrastructure Monitoring

In [18]:
# Initialize Cloudwatch
cloudwatch = boto3.client('cloudwatch', region_name='us-east-1')
endpoint_name = 'cardio-logistic-monitor-endpoint'
variant_name = 'AllTraffic'

# Alarm creation helper
def create_alarm(metric_name, threshold, comparison_operator='GreaterThanThreshold'):
    alarm_name = f"{endpoint_name}-{metric_name}-Alarm"

    cloudwatch.put_metric_alarm(
        AlarmName=alarm_name,
        MetricName=metric_name,
        Namespace='AWS/SageMaker',
        Dimensions=[
            {'Name': 'EndpointName', 'Value': endpoint_name},
            {'Name': 'VariantName', 'Value': variant_name}
        ],
        Statistic='Average',
        Period=300,  # 5 minutes
        EvaluationPeriods=1,
        Threshold=threshold,
        ComparisonOperator=comparison_operator,
        ActionsEnabled=False,  # Can be toggled on later
        AlarmDescription=f"Alarm when {metric_name} exceeds {threshold}",
        Unit='Percent'
    )
    print(f"Alarm created: {alarm_name}")

# Create alarms
create_alarm('CPUUtilization', threshold=70)
create_alarm('MemoryUtilization', threshold=75)
create_alarm('DiskUtilization', threshold=80)
create_alarm('Invocation5XXErrors', threshold=1, comparison_operator='GreaterThanOrEqualToThreshold')

Alarm created: cardio-logistic-monitor-endpoint-CPUUtilization-Alarm
Alarm created: cardio-logistic-monitor-endpoint-MemoryUtilization-Alarm
Alarm created: cardio-logistic-monitor-endpoint-DiskUtilization-Alarm
Alarm created: cardio-logistic-monitor-endpoint-Invocation5XXErrors-Alarm


### Save files into s3 bucket

In [19]:
s3 = boto3.client("s3")
bucket = "sagemaker-us-east-1-531690656306"
local_file = "cardio_data_and_infrastructure_monitors.ipynb"
s3_key = "cardio_project/cardio_data_and_infrastructure_monitors.ipynb"

try:
    s3.upload_file(local_file, bucket, s3_key)
    print("Upload successful:", f"s3://{bucket}/{s3_key}")
except Exception as e:
    print("Upload failed:", e)

Upload successful: s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_data_and_infrastructure_monitors.ipynb


In [20]:
bucket = 'sagemaker-us-east-1-531690656306'
prefix = 'cardio_project/'  # Adjust if needed

s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No files found in that prefix.")

cardio_project/
cardio_project/cardio_cloudwatch.ipynb
cardio_project/cardio_cloudwatch_and_data_reports.ipynb
cardio_project/cardio_cloudwatch_no_label.ipynb
cardio_project/cardio_data_and_infrastructure_monitors.ipynb
cardio_project/cardio_data_and_infrasturure_monitors.ipynb
cardio_project/cardio_data_quality_monitoring_schedule_v2.ipynb
cardio_project/cardio_data_split_v2.ipynb
cardio_project/cardio_data_split_v3.ipynb
cardio_project/cardio_delete_enpoint_and_monitoring_schedule.ipynb
cardio_project/cardio_delete_enpoint_and_monitoring_schedule.py
cardio_project/cardio_eda_and_feature_engineering.ipynb
cardio_project/cardio_inference_transform_job.ipynb
cardio_project/cardio_inference_transform_job_v2.ipynb
cardio_project/cardio_logistic_baseline.ipynb
cardio_project/cardio_logistic_baseline_complete.ipynb
cardio_project/cardio_logistic_baseline_v2.ipynb
cardio_project/cardio_model_evaluation_compare.ipynb
cardio_project/cardio_model_monitoring.ipynb
cardio_project/cardio_preproces

In [21]:
for prefix in ['model/', 'cardio_data/']:
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    print(f"\nContents of {prefix}:")
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("No files found.")


Contents of model/:
model/
model/cardio_prod_no_label.csv
model/inference.py
model/logistic_model.pkl
model/logistic_model.tar.gz

Contents of cardio_data/:
cardio_data/
cardio_data/baseline-results/constraints.json
cardio_data/baseline-results/statistics.json
cardio_data/cardio_cleaned.csv
cardio_data/cardio_engineered.csv
cardio_data/cardio_engineered_clean.csv
cardio_data/cardio_prod_no_label.csv
cardio_data/cardio_prod_split40.csv
cardio_data/cardio_test_split10.csv
cardio_data/cardio_train.csv
cardio_data/cardio_train_split40.csv
cardio_data/cardio_val_split10.csv
cardio_data/predictions/cardio_prod_no_label.csv.out
